In [1]:
import pandas as pd 
import sqlite3

In [2]:
conn = sqlite3.connect("../HomelessData.db")
c = conn.cursor()

In [6]:
coc_df = pd.read_sql("SELECT * FROM CocData", con = conn)
coc_df

,Coc,Total,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Year,CocYear,Unsheltered,Sheltered
0,AK-500,920,720,200,0,212,558,92,2020,AK-500 2020,55,1003
1,AK-501,965,690,275,0,47,405,0,2020,AK-501 2020,169,722
2,AL-500,888,545,309,34,235,1823,32,2020,AL-500 2020,247,601
3,AL-501,396,274,122,0,114,247,0,2020,AL-501 2020,364,267
4,AL-502,285,118,167,0,0,42,8,2020,AL-502 2020,41,174
...,...,...,...,...,...,...,...,...,...,...,...,...
2721,WV-500,117,77,40,0,1,18,0,2014,WV-500 2014,9,88
2722,WV-501,261,180,68,13,4,403,108,2014,WV-501 2014,10,208
2723,WV-503,430,302,128,0,2,178,0,2014,WV-503 2014,13,347
2724,WV-508,1157,804,353,0,66,620,0,2014,WV-508 2014,398,940


In [46]:
county_df = pd.read_sql("SELECT * FROM CountyData",con=conn)
county_df["Population"] = county_df["Population"].str.replace(",","")
county_df["Population"] = pd.to_numeric(county_df["Population"])

In [42]:
county_meta_df = pd.read_sql("SELECT * FROM CountyMeta",con=conn)
county_meta_df

,FIPS,county,state,Coc,Latitude,Longitude
0,1001,Autauga,AL,AL-507,32.536382,–86.644490
1,1003,Baldwin,AL,AL-501,30.659218,–87.746067
2,1005,Barbour,AL,AL-507,31.870670,–85.405456
3,1007,Bibb,AL,AL-507,33.015893,–87.127148
4,1009,Blount,AL,AL-503,33.977448,–86.567246
...,...,...,...,...,...,...
3124,56037,Sweetwater,WY,WY-500,41.660339,–108.875676
3125,56039,Teton,WY,WY-500,44.049321,–110.588102
3126,56041,Uinta,WY,WY-500,41.284726,–110.558947
3127,56043,Washakie,WY,WY-500,43.878831,–107.669052


In [49]:
# Merge the county data and county meta tables
new_county_df = pd.merge(county_df,county_meta_df,on="FIPS")
new_county_df

,FIPS,Population,Unemploment,year,county,state,Coc,Latitude,Longitude
0,1001,55869.0,2.7,2019,Autauga,AL,AL-507,32.536382,–86.644490
1,1001,55533.0,3.6,2018,Autauga,AL,AL-507,32.536382,–86.644490
2,1001,55390.0,4.0,2017,Autauga,AL,AL-507,32.536382,–86.644490
3,1001,55243.0,5.1,2016,Autauga,AL,AL-507,32.536382,–86.644490
4,1001,54864.0,5.2,2015,Autauga,AL,AL-507,32.536382,–86.644490
...,...,...,...,...,...,...,...,...,...
21912,56045,6968.0,4.1,2017,Weston,WY,WY-500,43.846213,–104.570020
21913,56045,7220.0,5.0,2016,Weston,WY,WY-500,43.846213,–104.570020
21914,56045,7208.0,3.4,2015,Weston,WY,WY-500,43.846213,–104.570020
21915,56045,7138.0,3.5,2014,Weston,WY,WY-500,43.846213,–104.570020


In [63]:
# group the counties by the Coc
grouped_df = new_county_df[["Population","Coc","year"]].groupby(["year","Coc"]).sum()
grouped_df["Unemployment"] = new_county_df[["Unemploment","Coc","year"]].groupby(["year","Coc"]).mean()["Unemploment"]
grouped_df.reset_index(inplace=True)
grouped_df

,year,Coc,Population,Unemployment
0,2013,AK-500,301142.0,5.200000
1,2013,AK-501,418032.0,10.191667
2,2013,AL-500,949114.0,5.900000
3,2013,AL-501,608803.0,7.700000
4,2013,AL-502,209007.0,7.925000
...,...,...,...,...
2704,2019,WV-500,122691.0,5.325000
2705,2019,WV-501,131347.0,4.800000
2706,2019,WV-503,264539.0,5.775000
2707,2019,WV-508,1273570.0,5.857778
